In [50]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

#   データAPI

#   RAM上にデータセット(データ要素のシーケンス)を作る
X = tf.range(10)    #   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
dataset = tf.data.Dataset.from_tensor_slices(X) #   10個の要素を含むデータセット


In [51]:
#dataset = dataset.repeat(3).batch(7)    #   repeat(3)で3回繰り返し、batch(7)で7個ずつのバッチにまとめる
                                        #   batch(7, drop_remainder=True)で端数バッチをのぞける
                                        #   detasetメソッドはデータを書き換えずに新しいdatasetを作る
                                        
#dataset = dataset.map(lambda x: x*2)  #   個々の要素を2倍する。適用する関数はtensorflow関数に変換できるものに限定される

#dataset = dataset.apply(tf.data.experimental.unbatch()) #   データセット全体に対する変換

#dataset = dataset.filter(lambda x: x<10)    #   10未満の要素にフィルタをかける

# for item in dataset.take(3):   #   take(n)はデータの一部(n個)のみを使う
#     print(item)

dataset = dataset.shuffle(buffer_size=1, seed=42).batch(7)  #   buffer_size=1ならシャッフルされない
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9], shape=(3,), dtype=int32)


In [ ]:
#   複数ファイルの行のインターリーブ
train_filepaths = "datasets/housing/my_train_*.csv"
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42) #   ファイルパスをシャッフルしたデータセットを得られる

n_readers = 5
dataset = filepath_dataset.interleave(  #   n_reader個ずつファイパスを抜き出し引数の関数を実行を繰り返し新しいデータセットを返す
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1), #   先頭行を読み飛ばして一行ずつ読む
    cycle_length=n_readers
)

#   データの前処理
X_mean, X_std = [...]   #   訓練セットの各特徴量の平均と標準偏差(教科書の例では要素数は8個)
n_inputs = 8

def preprocess(line):
    defs = [0.]*n_inputs + [tf.constant([], dtype=tf.float32)]  #   要素数n_inputs個のtf.float32型の空配列
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])   #   すべての要素を1次元配列につめこむ
    y = tf.stack(fields[-1:])   #   最後の要素だけ
    return (x - X_mean) / X_std, y  #   スケーリング


#   1つにまとめる
def csv_reader_dataset(filepaths, repeat=1, n_readers=5, 
                        n_read_threads=None, shuffle_buffer_size=10000,
                        n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_parse_threads  #   マルチスレッド化
    )
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)   #   マルチスレッド化
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)    #   prefetchによって常に1バッチ先を準備することにより処理速度を上げる

train_set = csv_reader_dataset(train_filepaths)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

model = keras.models.Sequential([...])
model.compile([...])
model.fit(train_set, epochs=10, validation_data=valid_set)  #   model.fitにdatasetを渡せる
model.evaluate(test_set)    #   model.evaluateにも渡せる
new_set = test_set.take(3).map(lambda X, y:X)
model.predict(new_set)

In [4]:
#   TFRecord形式 : データのロード、パースが訓練のボトルネックとなるときに有効
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")    #   バイナリ形式の書き込み
    f.write(b"And this is the second record")
    
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)    #   あとはcsvと同様に処理可能

#   TFRecordファイル形式の圧縮
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    [...]
    
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"], compression_type="GZIP")  #   ファイルの読み込み

#   プロトコルバッファ : TFRecordファイルの個々のレコードは通常プロトコルバッファ
# syntax = "proto3";
# message Person{
#     string name = 1;    #   バイナリ表現で使用されるfield IDを設定
#     int32 d = 2;
#     repeated string email = 3;
# }
#   from person_pb2 import Person
person = Person(name="Al", id=123, email=["a@b.com"])   #   Personを作る
person.name             #   "Al"
person.name = "Alice"   #   書き換え
person.email[0]         #   "a@b.com"
person.email.append("c@d.com")  #   追加
s = person.SerializeToString()  #   オブジェクトをバイト列にシリアライズ(一連の形式にする)。
person2 = Person()  #   新しいPerson
person2.ParseFromString(s)  #   sをパースできる.tf関数に組み込むにはtf.py_function()でラップする必要がある
person == person2   #   True



NameError: name 'tf' is not defined

In [ ]:
#   TFRecordでは主にExample protobufで表現される
# syntax = "proto3";
# message BytesList{ repeated bytes value = 1; }
# message FloatList{ repeated float value = 1 [packed = true]; }    #   [packed=true]でエンコードを効率化できるらしい
# message Int64List{ repeated int64 value = 1 [packed = true]; }
# message Feature{
#     oneof kind{ #   3つのリストのどれか
#         BytesList bytes_list = 1;
#         FloatList float_list = 2;
#         Int64List int64_list = 3;
#     }
# }
# message Features{ map<string, Feature> feature = 1; }
# message Example{ Features features = 1; }
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example
#   tf.train.Exampleを使ってTFRecordファイルに書き込む
person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"]))
            "id":   Feature(int64_list=Int64List(value=[123])),
            "emails":   Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }
    )
)
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

#   Exampleのロードとパース
feature_description = { #   記述辞書
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""), #   固定長特徴量
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails":   tf.io.VarLenFeature(tf.stirng)  #   emailsは可変
}
for serialized_example in tf.data.TFRecordDataset(["my_contats.tfrecord"]): #   ロード
    parsed_example = tf.io.parse_single_example(serialized_example, feature_description)   #   個々のExampleをパース


In [ ]:
#   SequenceExample protobufを使用したリスト処理
# message FeatureList{ repeated Feature feature = 1; }
# message FeatureLists{ map<string, FeatureList> feature_list = 1; }
# message SequenceExample{
#     Features context = 1;
#     FeatureLists feature_lists = 2;
# }

parsed_context, parsed_feature_lists = tf.io.parse_single_sequence_example( #   1つのSequenceExampleのパース
    serialized_sequence_example, context_feature_descriptions,
    sequence_feature_descriptions
)
parsed_content = tf.RaggedTensor.from_sparse(parsed_feature_lists["content"])

#   前処理の方法の一つとして前処理層を挿入する
means = np.mean(X_train, axis=0, keepdims=True)
stds = np.std(X_train, axis=0, keepdims=True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([
    keras.layers.Lambda(lambda inputs: (inputs-means)/(stds+eps)),
    [...]
])

class Standardization(keras.layers.Layer):  #   カスタムレイヤとしてもOK
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0,keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs-self.means_)/(self.stds_+keras.backend.epsilon())
std_layer = Standardization()
std_layer.adapt(data_sample)
model = keras.Sequential()
model.add(std_layer)
[...]   #   他レイヤ
model.compile([...])
model.fit([...])

In [1]:
#   ワンホットベクトルを使用したカテゴリ特徴量のエンコード : カテゴリ数が少ないときに有効
vocab = ["<1H OCEAN", "INRAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"] #   カテゴリのリスト
indices = tf.range(len(vocab), dtype=tf.int64)  #   インデックスに対応するテンソルを作る
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices) 
num_oov_buckets = 2    #   語彙外バケットの数を指定。カテゴリの数が変化するというときの対処法になりうる
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)  #   [3,5,1,1]
cat_one_hot = table.one_hot(cat_indices, depth=len(vocab)+num_oov_buckets)  #   ワンホットエンコーディング。[[0,0,0,1,0,0,0],[...]]

#   埋め込み(ベクトルを使ってカテゴリを表現する方法)を使用したカテゴリ特徴量のエンコード : カテゴリ数が多いときに有効
embedding_dim = 2   #   2次元ベクトル
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)  #   6*2行列
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
tf.nn.embedding_lookup(embedding_matrix, cat_indices)   #   4*2行列。4つのカテゴリに対応する2次元ベクトル
embedding = keras.layers.Embedding(input_dim=len(vocab)+num_oov_buckets, output_dim=embedding_dim)  #   埋め込み行列を処理するEmbedding層
embedding(cat_indices)
#   埋め込み学習
regular_inputs = keras.layers.Input(shape=[8])  #   数値特徴量を含む通常入力
categories = keras.layers.Input(shape=[], dtype=string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)  #   カテゴリ入力
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])  #   以上を連結
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[regular_inputs, categories], outputs=[outputs])


NameError: name 'tf' is not defined

In [ ]:
#   keras前処理層の詳細
normalization = keras.layers.Normalization()    #   正規化
discretization = keras.layers.Discretization([...]) #   離散化
pipeline = keras.layers.PreprocessingStage([normalization, discretization])    #   複数の前処理層をつなげることができる
pipeline.adapt(data_sample)

#   Apache Beamによる効率上昇、TF Transformによる保守の簡易化
import tensorflow_transform as tft
def preprocess(inputs): #   Featuresの入力バッチ
    median_age = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]
    standardized_age = tft.scale_to_z_score(median_age)
    ocean_proximity_id = tft.compute_and_apply_vocabulary(ocean_proximity)
    return {
        "standardized_median_age": standardized_age,
        "ocean_proximity_id": ocean_proximity_id
    }

#   MNISTなど標準データセットが使えればいいのであればTFDSを使う
import tensorflow_datasets as tfds

dataset  = tfds.load(name="mnist")  #   データのダウンロード.特徴量とラベルの辞書になっている
mnist_train, mnist_test = dataset["train"], dataset["test"]
mnist_train = mnist_train.shuffle(10000).batch(32).prefetch(1)
for item in mnist_train:
    images = item["image"]
    labels = item["label"]
    [...]
    
mnist_train = mnist_train.shuffle(10000).batch(32)
mnist_train = mnist_train.map(lambda items: (items["image"], items["label"]))   #   kerasは個々の要素が2要素tupleとなることを前提としているので、
                                                                                #   2要素tupleに変換
mnist_train = mnist_train.prefetch(1)

#   上の方法よりも簡単な方法
dataset = tfds.load(name="mnist", batch_size=32, as_supervised=True)    #   as_supervised=Trueによって変換する
mnist_train = dataset["train"].prefetch(1)
model = keras.Sequential([...])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
model.fit(mnist_train, epoch=5)